In [2]:
import pandas as pd


In [7]:
data = pd.read_csv('./data/sample-data.csv', header=None, on_bad_lines='skip')
data.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,address,price,gross_tax,strata_maintenance_fees,bedrooms,bathrooms,property_type,property_age,title,style,...,appliances,community,days_on_rew,property_views,mls®_number,source,frontage,lot_size,year_built,depth
1,3735 Puget Drive,"$5,498,800","$19,041",NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,57 ft x 122 ft (6954 ft²),built in 1987 (35 yrs old),122
2,3735 Puget Drive,"$5,498,800","$19,041",NaN,3,4,house,NaN,freehold nonstrata,2 storey w/bsmt.,...,NaN,arbutus,13 days,1491,r2735110,rebgv,57.00 feet,57 ft x 122 ft (6954 ft²),built in 1987 (35 yrs old),122
3,302 5118 Cambie Street,"$2,367,900",$0,$925,3,3,apt/condo,NaN,freehold strata,1 storey,...,"in suite laundry, microwave, range top, washer...",cambie,109 days,592,r2712600,rebgv,NaN,NaN,built in 2022 (0 yrs old),NaN
4,438 W 17th Avenue,"$3,888,000","$10,064",NaN,8,4,multifamily,NaN,freehold nonstrata,NaN,...,NaN,cambie,61 days,1129,r2722725,rebgv,49.70 feet,49 ft x 137 ft (6807 ft²),built in 1930 (92 yrs old),137.5


In [6]:
houses = data[data[6] == 'House']
houses.drop(houses.columns[15:], axis=1, inplace=True)
house_columns = ["address",
"price",
"gross_tax",
"strata_maintenance_fees",
"bedrooms",
"bathrooms",
"property_type",
"property_age",
"title",
"style",
"heating_type",
"feature",
"amenities",
"appliances",
"community",
"days_on_rew",
"property_views",
"mls®_number",
"source",
'frontage', 
'lot_size', 
'year_built', 
'depth']
houses.columns = house_columns
houses.head()

/var/folders/hx/wsc86f0s71j8zp38fr7x77pc0000gn/T/ipykernel_25285/2287298755.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses.drop(houses.columns[15:], axis=1, inplace=True)


ValueError: Length mismatch: Expected axis has 15 elements, new values have 23 elements

In [ ]:
# Filter Houses Only

: 

In [ ]:
# Convert Price column to integer

: 

In [ ]:
# convert Lot Size to Integer

: 

In [ ]:
# Convert Gross Tax Income to Integer

: 

In [ ]:
# Convert House Age to integer - Consider prebuild houses with age 0

: 